<a href="https://colab.research.google.com/github/JiaChangGit/NeuroCuts_robin/blob/colab/Neurocuts_note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


前置準備
環境 : Colab https://colab.research.google.com/
1.掛接雲端硬碟
目的是為了讓訓練的結果，能夠存放在自己的雲端
因為colab免費版和基本付費版，最多只能連續使用24hr，之後就會洗掉重置，只有高階付費版才不會重置

2.將python降版至3.7，安裝tool，並且clone github



```
# 此內容會顯示為程式碼
#-----To install specific version 3.7 of python in Google Colab-----
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3 #按有注音的數字鍵(選3.7)
!python --version
!sudo apt install python3-pip
!sudo apt install python3.7-distutils
!python -m pip install --upgrade pip
!pip install tensorflow==1.14.0
!pip install ray==0.7.6
!pip install gym==0.17.3
!pip install opencv-python
!pip install setproctitle
!pip install tabulate
!pip install requests
!pip install psutil
!pip install lz4
!pip uninstall protobuf
!pip install protobuf==3.19.0
!git clone https://github.com/ourobin2/NeuroCuts_robin.git
%cd NeuroCuts_robin/
%ls
```

模型訓練
1.指令說明
--rules=[要訓練的ruleset]
--dump-dir=[訓練好的tree要存的路徑]
--depth-weight=[w的值，預設為1]
--partition-mode=efficuts[root node模式，預設為None]

論文中的實驗使用了4種不同的版本
若欲運行原方法(版本3、版本4)，需更改程式碼

**版本(1) proposed**

```
# 此內容會顯示為程式碼
#example
!python run_neurocuts.py --rules=ipc2_100k  --dump-dir=/content/drive/MyDrive/CIAL/proposed_internal_range_plus1_noefficut/w=0.99999,u=0/ipc2_100k --depth-weight=0.99999
```

版本(2) proposed+efficut

```
# 此內容會顯示為程式碼
#example
!python run_neurocuts.py --rules=ipc2_100k  --dump-dir=/content/drive/MyDrive/CIAL/neurocut_update_efficut/w=1,u=0/ipc2_100k --depth-weight=1 --partition-mode=efficuts  
```

版本(3) efficut

```
# 此內容會顯示為程式碼
#example
!python run_neurocuts.py --rules=ipc2_100k  --dump-dir=/content/drive/MyDrive/CIAL/neurocut_update_efficut/w=1,u=0/ipc2_100k --depth-weight=1 --partition-mode=efficuts  
```

版本(4) Neurocuts

```
# 此內容會顯示為程式碼
#example
!python run_neurocuts.py --rules=ipc2_100k  --dump-dir=/content/drive/MyDrive/CIAL/neurocut_update/w=1,u=0/ipc2_100k --depth-weight=1
```

2.運行模型

若是100k的模型 平均需要16~22小時才會收斂
若只是要測試，可以改跑1k的模型，並且設定為快速模式
會把memory access, update memory access, bytes_per_rule當下最好的存下來

可以至vm查看訓練詳細資料
舉例路徑: /root/ray_results/neurocuts_efficuts/PPO_tree_env_0_rules=_content_NeuroCuts_robin_ruleset_fw5_100k,tree_gae_lambda=0.95_2023-07-12_05-57-30ab7gapzh/result.json
如有研究需求也可以把上述的result.json檔存至雲端

想要查看tree的架構可以使用 inspect_tree.py 指令

```
# 此內容會顯示為程式碼
!python inspect_tree.py /content/drive/MyDrive/CIAL/proposed_internal_range_plus1/w=0.99999,u=0/acl2_100k/acl2_100k-55-acc-17590-uacc-27.37-bytes-1688528736.69055.pkl
```

如果只是要測試模擬的表現(非cycle time)，到此階段即結束


Cycle time測試
1.選擇欲測試之tree
2.pkl檔轉換成txt檔
指令: !python pickle-to-json.py [.pkl路徑]

```
# 此內容會顯示為程式碼
!python pickle-to-json.py /content/drive/MyDrive/CIAL/proposed_internal_range_plus1_noefficut/w=0.99999,u=0/acl5_100k/acl5_100k-6-acc-15594-uacc-17.08-bytes-1688327649.3373015.pkl
```
3.會在/content/NeuroCuts_robin 路徑下產生test.txt
4.移至run cycle time的環境 (我是在178)
5.把test.txt重新命名，並移到該環境
6.編譯+執行 ./partition [ruleset] [.txt] [trace]

```
# 此內容會顯示為程式碼
//example
gcc partition_time.c -lm -o partition
./partition ruleset/acl1_100k tree/largeness_acl1_100k_41_21.99.txt trace/acl1_100k_trace
```
7.便能夠產生search time 結果




運行原方法(版本3、版本4) 步驟

neurocuts_env.py

```
# 此內容會顯示為程式碼
import collections
import time
import os
import numpy as np
import pickle
from gym.spaces import Tuple, Box, Discrete, Dict

from ray.rllib.env import MultiAgentEnv
from ray.rllib.evaluation.rollout_worker import get_global_worker

from tree import Tree, load_rules_from_file
from hicuts import HiCuts

NUM_DIMENSIONS = 5
NUM_PART_LEVELS = 6  # 2%, 4%, 8%, 16%, 32%, 64%


class NeuroCutsEnv(MultiAgentEnv):
    """NeuroCuts multi-agent tree building environment.

    In this env, each "cut" in the tree is an action taken by a
    different agent. All the agents share the same policy. We
    aggregate rewards at the end of the episode and assign each
    cut its reward based on the policy performance (actual depth).
    """

    def __init__(self,
                 rules_file,
                 leaf_threshold=16,
                 max_cuts_per_dimension=5,
                 max_actions_per_episode=5000,
                 max_depth=100,
                 partition_mode=None,
                 reward_shape="linear",
                 depth_weight=1.0,
                 update_weight = 0, ##
                 dump_dir=None,
                 tree_gae=True,
                 tree_gae_gamma=1.0,
                 tree_gae_lambda=0.95,
                 zero_obs=False):

        self.tree_gae = tree_gae
        self.tree_gae_gamma = tree_gae_gamma
        self.tree_gae_lambda = tree_gae_lambda
        self.reward_shape = {
            "linear": lambda x: x,
            "log": lambda x: np.log(x),
        }[reward_shape]
        self.zero_obs = zero_obs

        assert partition_mode in [None, "simple", "efficuts", "cutsplit"]
        self.partition_enabled = partition_mode == "simple"
        if partition_mode in ["efficuts", "cutsplit"]:
            self.force_partition = partition_mode
        else:
            self.force_partition = False

        self.dump_dir = dump_dir and os.path.expanduser(dump_dir)
        if self.dump_dir:
            try:
                os.makedirs(self.dump_dir)
            except:
                pass
        self.best_time = float("inf")
        self.best_space = float("inf")
        self.best_utime = float("inf") ##

        self.depth_weight = depth_weight
        self.update_weight = update_weight ##
        self.rules_file = rules_file
        self.rules = load_rules_from_file(rules_file)
        self.leaf_threshold = leaf_threshold
        self.max_actions_per_episode = max_actions_per_episode
        self.max_depth = max_depth
        self.num_actions = None
        self.tree = None
        self.node_map = None
        self.child_map = None
        self.max_cuts_per_dimension = max_cuts_per_dimension
        if self.partition_enabled:
            self.num_part_levels = NUM_PART_LEVELS
        else:
            self.num_part_levels = 0
        self.action_space = Tuple([
            Discrete(NUM_DIMENSIONS),
            Discrete(max_cuts_per_dimension + self.num_part_levels)
        ])
        self.observation_space = Dict({
            "real_obs": Box(0, 99999999, (279, ), dtype=np.float32),
            "action_mask": Box(
                0,
                1, (NUM_DIMENSIONS + max_cuts_per_dimension + self.num_part_levels, ),
                dtype=np.float32),
        })

    def reset(self):
        self.num_actions = 0
        self.exceeded_max_depth = []
        self.tree = Tree(
            self.rules,
            self.leaf_threshold,
            refinements={
                "node_merging": True,
                "rule_overlay": True,
                "region_compaction": False,
                "rule_pushup": False,
                "equi_dense": False,
            })
        self.node_map = {
            self.tree.root.id: self.tree.root,
        }
        self.child_map = {}

        if self.force_partition:
            if self.force_partition == "cutsplit":
                self.tree.partition_cutsplit()
            elif self.force_partition == "efficuts":
                self.tree.partition_efficuts()
            else:
                assert False, self.force_partition
            for c in self.tree.root.children:
                self.node_map[c.id] = c
            self.child_map[self.tree.root.id] = [
                c.id for c in self.tree.root.children
            ]

        start = self.tree.current_node
        return {start.id: self._encode_state(start)}

    def step(self, action_dict):
        assert len(action_dict) == 1  # one at a time processing

        new_children = []
        for node_id, action in action_dict.items():
            node = self.node_map[node_id]
            orig_action = action
            if np.isscalar(action):
                assert not self.partition_enabled, action
                partition = False
                cut_dimension = int(action) % 5
                cut_num = int(action) // 5
                action = [cut_dimension, cut_num]
            else:
                if action[1] >= self.max_cuts_per_dimension:
                    assert self.partition_enabled, (
                        action, self.max_cuts_per_dimension)
                    partition = True
                    action[1] -= self.max_cuts_per_dimension
                else:
                    partition = False

            if partition:
                children = self.tree.partition_node(node, action[0], action[1])
            else:
                cut_dimension, cut_num = self.action_tuple_to_cut(node, action)
                children = self.tree.cut_node(node, cut_dimension,
                                              int(cut_num))

            self.num_actions += 1
            num_leaf = 0
            for c in children:
                self.node_map[c.id] = c
                if not self.tree.is_leaf(c):
                    new_children.append(c)
                else:
                    num_leaf += 1
            self.child_map[node_id] = [c.id for c in children]

        node = self.tree.get_current_node()
        while node and (self.tree.is_leaf(node)
                        or node.depth > self.max_depth):
            node = self.tree.get_next_node()
            if node and node.depth > self.max_depth:
                self.exceeded_max_depth.append(node)
        nodes_remaining = self.tree.nodes_to_cut + self.exceeded_max_depth

        obs, rew, done, info = {}, {}, {}, {}

        if (not nodes_remaining
                or self.num_actions > self.max_actions_per_episode
                or self.tree.get_current_node() is None):
            zero_state = self._zeros()
            rew = self.compute_rewards(self.depth_weight,self.update_weight) ##
            stats = {}
            obs = {node_id: zero_state for node_id in rew.keys()}
            if self.tree_gae:
                advantages, stats = self.compute_gae(self.depth_weight)
                info = {
                    node_id: {
                        "__advantage__": advantages[node_id],
                        "__value_target__": rew[node_id],
                    }
                    for node_id in rew.keys()
                }
            else:
                info = {node_id: {} for node_id in rew.keys()}
            result = self.tree.compute_result()
            rules_remaining = set()
            for n in nodes_remaining:
                for r in n.rules:
                    rules_remaining.add(str(r))
            info[self.tree.root.id].update({
                "bytes_per_rule": result["bytes_per_rule"],
                "memory_access": result["memory_access"],
                "update_memory_access": result["update_memory_access"],
                "exceeded_max_depth": len(self.exceeded_max_depth),
                "tree_depth": self.tree.get_depth(),
                "tree_stats": self.tree.get_stats(),
                "tree_stats_str": self.tree.stats_str(),
                "nodes_remaining": len(nodes_remaining),
                "rules_remaining": len(rules_remaining),
                "num_nodes": len(self.node_map),
                "partition_fraction": float(
                    len([
                        n for n in self.node_map.values() if n.is_partition()
                    ])) / len(self.node_map),
                "num_splits": self.num_actions,
                "rules_file": self.rules_file,
            })
            info[self.tree.root.id].update(stats)
            if not nodes_remaining and self.dump_dir:
                self.save_if_best(result)
            return obs, rew, {"__all__": True}, info

        needs_split = [self.tree.get_current_node()]
        obs.update({s.id: self._encode_state(s) for s in needs_split})
        rew.update({s.id: 0 for s in needs_split})
        done.update({"__all__": False})
        info.update({s.id: {} for s in needs_split})
        return obs, rew, done, info

    def save_if_best(self, result):
        time_stat = int(result["memory_access"])
        space_stat = round(float(result["bytes_per_rule"]),2)
        update_stat = int(result["update_memory_access"])
        save = False
        if time_stat < self.best_time:
            self.best_time = time_stat
            save = True
        if space_stat < self.best_space:
            self.best_space = space_stat
            save = True
        if update_stat < self.best_utime:
            self.best_utime = update_stat
            save = True
        if save:
            out = os.path.join(
                self.dump_dir, "{}-{}-acc-{}-uacc-{}-bytes-{}.pkl".format(
                    os.path.basename(self.rules_file), time_stat, update_stat, space_stat,
                    time.time()))
            print("Saving tree to {}".format(out))
            with open(out, "wb") as f:
                pickle.dump(self.tree, f)

    def action_tuple_to_cut(self, node, action):
        cut_dimension = action[0]
        range_left = node.ranges[cut_dimension * 2]
        range_right = node.ranges[cut_dimension * 2 + 1]
        cut_num = max(2, min(2**(action[1] + 1), range_right - range_left))
        return (cut_dimension, cut_num)

    def compute_gae(self, depth_weight):
        """Compute GAE for a branching decision environment.

           V(d) = min over nodes n at depth=d V(n)
        """

        assert depth_weight == 1.0, "GAE not supported with space weight"

        # First precompute the value of each node
        V = {}
        stats = {}
        ev = get_global_worker()
        assert ev.policy_config["use_gae"], ev.policy_config["use_gae"]
        assert ev.policy_config["lambda"] == 1.0, ev.policy_config["lambda"]
        policy = ev.get_policy()
        prep = ev.preprocessors["default_policy"]
        nlist = list(self.node_map.items())
        feed_dict = {
            policy.get_placeholder("obs"): [
                prep.transform(self._encode_state(node)) for (_, node) in nlist
            ],
            policy.get_placeholder("prev_actions"): [[0, 0] for _ in nlist],
            policy.get_placeholder("prev_rewards"): [0.0 for _ in nlist],
            policy.model.seq_lens: [1 for _ in nlist],
        }
        vf = policy.sess.run(policy.value_function, feed_dict)
        V_root = 0.0
        for (node_id, _), v in zip(nlist, list(vf)):
            V[node_id] = v
            if node_id == self.tree.root.id:
                V_root = v

#        print(
#            "Computed node values",
#            "mean", np.mean(vf), "min", np.min(vf), "max", np.max(vf),
#            "count", len(vf))
        stats["V_gae_min"] = float(np.min(vf))
        stats["V_gae_max"] = float(np.max(vf))
        stats["V_gae_mean"] = float(np.mean(vf))
        stats["V_gae_root"] = float(V_root)

        gamma = self.tree_gae_gamma
        lambd = self.tree_gae_lambda

        # Map from node_id -> depth -> min(V at depth)
        # These values are unique per (node_id, depth) combination
        min_V_for_node = collections.defaultdict(dict)

        # Then, compute the min V at each level for each subtree
        incomplete = True
        while incomplete:
            incomplete = False
            for node_id, node in self.node_map.items():
                if node_id in min_V_for_node:
                    continue

                children = self.child_map.get(node_id, [])
                if self.tree.is_leaf(node):
                    min_V_for_node[node_id][node.depth] = -1
                elif not children:
                    min_V_for_node[node_id][node.depth] = V[node_id]
                elif all((c_id in min_V_for_node) for c_id in children):
                    min_V = {}
                    for c_id in children:
                        for depth, minv in min_V_for_node[c_id].items():
                            if depth not in min_V:
                                min_V[depth] = minv
                            else:
                                min_V[depth] = min(min_V[depth], minv)
                    assert node.depth not in min_V, min_V
                    min_V[node.depth] = V[node_id]
                    min_V_for_node[node_id] = min_V
                else:
                    incomplete = True
                    continue
#                print(
#                    "Computed minV for node", node_id, "depth", node.depth,
#                    min_V_for_node[node_id])

# delta(V)_{t+1} in the GAE paper

        def deltaV(node_id, depth):
            dv = -1 + gamma * min_V_for_node[node_id].get(d + 1, 0.0)
            dv -= min_V_for_node[node_id][d]
            return dv

        # Now we can compute GAE estimates for each
        advantages = {}
        adv_list = []
        adv_root = 0.0
        for node_id, node in self.node_map.items():
            A_gae = 0.0
            d = node.depth
            while d in min_V_for_node[node_id]:
                A_gae += (gamma * lambd)**(d - node.depth) * deltaV(node_id, d)
                d += 1
#            print("A_gae for node", node_id, "depth", node.depth, A_gae)
            adv_list.append(A_gae)
            if node_id == self.tree.root.id:
                adv_root = A_gae
            advantages[node_id] = A_gae


#        print(
#            "GAE advantages",
#            "min", np.min(adv_list), "max", np.max(adv_list),
#            "mean", np.mean(adv_list))
        stats["A_gae_min"] = float(np.min(adv_list))
        stats["A_gae_max"] = float(np.max(adv_list))
        stats["A_gae_mean"] = float(np.mean(adv_list))
        stats["A_gae_root"] = float(adv_root)

        return advantages, stats

    def compute_rewards(self, depth_weight, update_weight):
        depth_to_go = collections.defaultdict(int)
        nodes_to_go = collections.defaultdict(int)
        updates_to_go = collections.defaultdict(int) ##
        num_updates = 1
        while num_updates > 0:
            num_updates = 0
            for node_id, node in self.node_map.items():
                if node_id not in depth_to_go:
                    if self.tree.is_leaf(node):
                        depth_to_go[node_id] = 0
                        nodes_to_go[node_id] = 0
                        updates_to_go[node_id] = 0 ##
                    else:
                        depth_to_go[node_id] = 1
                        nodes_to_go[node_id] = 1
                        updates_to_go[node_id] = 1 ##
                if node_id in self.child_map:
                    if self.node_map[node_id].is_partition():
                        max_child_depth = self.tree_gae_gamma * sum(
                            [depth_to_go[c] for c in self.child_map[node_id]])
                        max_child_update = 1+ self.tree_gae_gamma * max(
                            [updates_to_go[c] for c in self.child_map[node_id]]) ##
                    else:
                        max_child_depth = 1 + self.tree_gae_gamma * max(
                            [depth_to_go[c] for c in self.child_map[node_id]])
                        max_child_update = self.tree_gae_gamma * sum(
                            [updates_to_go[c] for c in self.child_map[node_id]]) ##
                    if max_child_depth > depth_to_go[node_id]: ##
                        depth_to_go[node_id] = max_child_depth ##
                        num_updates += 1 ##
                    if max_child_update > updates_to_go[node_id]:
                        updates_to_go[node_id] = max_child_update
                        num_updates += 1
                    sum_child_cuts = len(self.child_map[node_id]) + sum(
                        [nodes_to_go[c] for c in self.child_map[node_id]])
                    if sum_child_cuts > nodes_to_go[node_id]:
                        nodes_to_go[node_id] = sum_child_cuts
                        num_updates += 1
        rew = {
            node_id:
            -depth_weight * self.reward_shape(depth) - update_weight * self.reward_shape(float(updates_to_go[node_id])) -  (1.0 - depth_weight- update_weight) *
            self.reward_shape(float(nodes_to_go[node_id]))
            for (node_id, depth) in depth_to_go.items()
            if node_id in self.child_map
        }
        return rew

    def _zeros(self):
        zeros = np.array([0] * 279)
        return {
            "real_obs": zeros,
            "action_mask": np.array([1] *
                (5 + self.max_cuts_per_dimension + self.num_part_levels)),
        }

    def _encode_state(self, node):
        if node.depth > 1:
            action_mask = ([1] * (NUM_DIMENSIONS + self.max_cuts_per_dimension) +
                           [0] * self.num_part_levels)
        else:
            assert node.depth == 1, node.depth
            action_mask = ([1] * (NUM_DIMENSIONS + self.max_cuts_per_dimension)
                           + [1] * self.num_part_levels)
        s = np.array(node.get_state())
        return {
            "real_obs": np.zeros_like(s) if self.zero_obs else s,
            "action_mask": np.array(action_mask),
        }
```

---

tree.py

```
# 此內容會顯示為程式碼
import math
import random
import numpy as np
import re
import sys

sys.setrecursionlimit(99999)
SPLIT_CACHE = {}


class Rule:
    def __init__(self, priority, ranges):
        # each range is left inclusive and right exclusive, i.e., [left, right)
        self.priority = priority
        self.ranges = ranges
        self.names = ["src_ip", "dst_ip", "src_port", "dst_port", "proto"]

    def is_intersect(self, dimension, left, right):
        return not (left >= self.ranges[dimension*2+1] or \
            right <= self.ranges[dimension*2])

    def is_intersect_multi_dimension(self, ranges):
        for i in range(5):
            if ranges[i*2] >= self.ranges[i*2+1] or \
                    ranges[i*2+1] <= self.ranges[i*2]:
                return False
        return True

    def sample_packet(self):
        src_ip = random.randint(self.ranges[0], self.ranges[1] - 1)
        dst_ip = random.randint(self.ranges[2], self.ranges[3] - 1)
        src_port = random.randint(self.ranges[4], self.ranges[5] - 1)
        dst_port = random.randint(self.ranges[6], self.ranges[7] - 1)
        protocol = random.randint(self.ranges[8], self.ranges[9] - 1)
        packet = (src_ip, dst_ip, src_port, dst_port, protocol)
        assert self.matches(packet), packet
        return packet

    def matches(self, packet):
        assert len(packet) == 5, packet
        return self.is_intersect_multi_dimension([
            packet[0] + 0,  # src ip
            packet[0] + 1,
            packet[1] + 0,  # dst ip
            packet[1] + 1,
            packet[2] + 0,  # src port
            packet[2] + 1,
            packet[3] + 0,  # dst port
            packet[3] + 1,
            packet[4] + 0,  # protocol
            packet[4] + 1
        ])

    def is_covered_by(self, other, ranges):
        for i in range(5):
            if (max(self.ranges[i*2], ranges[i*2]) < \
                    max(other.ranges[i*2], ranges[i*2]))or \
                    (min(self.ranges[i*2+1], ranges[i*2+1]) > \
                    min(other.ranges[i*2+1], ranges[i*2+1])):
                return False
        return True

    def __str__(self):
        result = ""
        for i in range(len(self.names)):
            result += "%s:[%d, %d) " % (self.names[i], self.ranges[i * 2],
                                        self.ranges[i * 2 + 1])
        return result


def load_rules_from_file(file_name):
    rules = []
    rule_fmt = re.compile(r'^@(\d+).(\d+).(\d+).(\d+)/(\d+) '\
        r'(\d+).(\d+).(\d+).(\d+)/(\d+) ' \
        r'(\d+) : (\d+) ' \
        r'(\d+) : (\d+) ' \
        r'(0x[\da-fA-F]+)/(0x[\da-fA-F]+) ' \
        r'(.*?)')
    for idx, line in enumerate(open(file_name)):
        elements = line[1:-1].split('\t')
        line = line.replace('\t', ' ')

        sip0, sip1, sip2, sip3, sip_mask_len, \
        dip0, dip1, dip2, dip3, dip_mask_len, \
        sport_begin, sport_end, \
        dport_begin, dport_end, \
        proto, proto_mask = \
        (eval(rule_fmt.match(line).group(i)) for i in range(1, 17))

        sip0 = (sip0 << 24) | (sip1 << 16) | (sip2 << 8) | sip3
        sip_begin = sip0 & (~((1 << (32 - sip_mask_len)) - 1))
        sip_end = sip0 | ((1 << (32 - sip_mask_len)) - 1)

        dip0 = (dip0 << 24) | (dip1 << 16) | (dip2 << 8) | dip3
        dip_begin = dip0 & (~((1 << (32 - dip_mask_len)) - 1))
        dip_end = dip0 | ((1 << (32 - dip_mask_len)) - 1)

        if proto_mask == 0xff:
            proto_begin = proto
            proto_end = proto
        else:
            proto_begin = 0
            proto_end = 0xff

        rules.append(
            Rule(idx, [
                sip_begin, sip_end + 1, dip_begin, dip_end + 1, sport_begin,
                sport_end + 1, dport_begin, dport_end + 1, proto_begin,
                proto_end + 1
            ]))
    return rules


def to_bits(value, n):
    if value >= 2**n:
        print("WARNING: clamping value", value, "to", 2**n - 1)
        value = 2**n - 1
    assert value == int(value)
    b = list(bin(int(value))[2:])
    assert len(b) <= n, (value, b, n)
    return [0.0] * (n - len(b)) + [float(i) for i in b]


def onehot_encode(arr, n):
    out = []
    for a in arr:
        x = [0] * n
        for i in range(a):
            x[i] = 1
        out.extend(x)
    return out


class Node:
    def __init__(self, id, ranges, rules, depth, partitions, manual_partition):
        self.id = id
        self.partitions = list(partitions or [])
        self.manual_partition = manual_partition
        self.ranges = ranges
        self.rules = rules
        self.depth = depth
        self.children = []
        self.action = None
        self.pushup_rules = None
        self.num_rules = len(self.rules)

    def is_partition(self):
        """Returns if node was partitioned."""
        if not self.action:
            return False
        elif self.action[0] == "partition":
            return True
        elif self.action[0] == "cut":
            return False
        else:
            return False

    def match(self, packet):
        if self.is_partition():
            matches = []
            for c in self.children:
                match = c.match(packet)
                if match:
                    matches.append(match)
            if matches:
                matches.sort(key=lambda r: self.rules.index(r))
                return matches[0]
            return None
        elif self.children:
            for n in self.children:
                if n.contains(packet):
                    return n.match(packet)
            return None
        else:
            for r in self.rules:
                if r.matches(packet):
                    return r

    def is_intersect_multi_dimension(self, ranges):
        for i in range(5):
            if ranges[i*2] >= self.ranges[i*2+1] or \
                    ranges[i*2+1] <= self.ranges[i*2]:
                return False
        return True

    def contains(self, packet):
        assert len(packet) == 5, packet
        return self.is_intersect_multi_dimension([
            packet[0] + 0,  # src ip
            packet[0] + 1,
            packet[1] + 0,  # dst ip
            packet[1] + 1,
            packet[2] + 0,  # src port
            packet[2] + 1,
            packet[3] + 0,  # dst port
            packet[3] + 1,
            packet[4] + 0,  # protocol
            packet[4] + 1
        ])

    def is_useless(self):
        if not self.children:
            return False
        return max(len(c.rules) for c in self.children) == len(self.rules)

    def pruned_rules(self):
        new_rules = []
        for i in range(len(self.rules) - 1):
            rule = self.rules[len(self.rules) - 1 - i]
            flag = False
            for j in range(0, len(self.rules) - 1 - i):
                high_priority_rule = self.rules[j]
                if rule.is_covered_by(high_priority_rule, self.ranges):
                    flag = True
                    break
            if not flag:
                new_rules.append(rule)
        new_rules.append(self.rules[0])
        new_rules.reverse()
        return new_rules

    def get_state(self):
        state = []
        state.extend(to_bits(self.ranges[0], 32))
        state.extend(to_bits(self.ranges[1] - 1, 32))
        state.extend(to_bits(self.ranges[2], 32))
        state.extend(to_bits(self.ranges[3] - 1, 32))
        assert len(state) == 128, len(state)
        state.extend(to_bits(self.ranges[4], 16))
        state.extend(to_bits(self.ranges[5] - 1, 16))
        state.extend(to_bits(self.ranges[6], 16))
        state.extend(to_bits(self.ranges[7] - 1, 16))
        assert len(state) == 192, len(state)
        state.extend(to_bits(self.ranges[8], 8))
        state.extend(to_bits(self.ranges[9] - 1, 8))
        assert len(state) == 208, len(state)

        if self.manual_partition is None:
            # 0, 6 -> 0-64%
            # 6, 7 -> 64-100%
            partition_state = [
                0,
                7,  # [>=min, <max) -- 0%, 2%, 4%, 8%, 16%, 32%, 64%, 100%
                0,
                7,
                0,
                7,
                0,
                7,
                0,
                7,
            ]
            for (smaller, part_dim, part_size) in self.partitions:
                if smaller:
                    partition_state[part_dim * 2 + 1] = min(
                        partition_state[part_dim * 2 + 1], part_size + 1)
                else:
                    partition_state[part_dim * 2] = max(
                        partition_state[part_dim * 2], part_size + 1)
            state.extend(onehot_encode(partition_state, 7))
        else:
            partition_state = [0] * 70
            partition_state[self.manual_partition] = 1
            state.extend(partition_state)
        state.append(self.num_rules)
        return np.array(state)

    def __str__(self):
        result = "ID:%d\tAction:%s\tDepth:%d\tRange:\t%s\nChildren: " % (
            self.id, str(self.action), self.depth, str(self.ranges))
        for child in self.children:
            result += str(child.id) + " "
        result += "\nRules:\n"
        for rule in self.rules:
            result += str(rule) + "\n"
        if self.pushup_rules != None:
            result += "Pushup Rules:\n"
            for rule in self.pushup_rules:
                result += str(rule) + "\n"
        return result


class Tree:
    def __init__(
            self,
            rules,
            leaf_threshold,
            refinements={
                "node_merging": False,
                "rule_overlay": False,
                "region_compaction": False,
                "rule_pushup": False,
                "equi_dense": False
            }):
        # hyperparameters
        self.leaf_threshold = leaf_threshold
        self.refinements = refinements

        self.rules = rules
        self.root = self.create_node(
            0, [0, 2**32, 0, 2**32, 0, 2**16, 0, 2**16, 0, 2**8], rules, 1,
            None, None)
        if (self.refinements["region_compaction"]):
            self.refinement_region_compaction(self.root)
        self.current_node = self.root
        self.nodes_to_cut = [self.root]
        self.depth = 1
        self.node_count = 1

    def create_node(self, id, ranges, rules, depth, partitions,
                    manual_partition):
        node = Node(id, ranges, rules, depth, partitions, manual_partition)

        if self.refinements["rule_overlay"]:
            self.refinement_rule_overlay(node)

        return node

    def match(self, packet):
        return self.root.match(packet)

    def get_depth(self):
        return self.depth

    def get_current_node(self):
        return self.current_node

    def is_leaf(self, node):
        return len(node.rules) <= self.leaf_threshold

    def is_finish(self):
        return len(self.nodes_to_cut) == 0

    def update_tree(self, node, children):
        if self.refinements["node_merging"]:
            children = self.refinement_node_merging(children)

        if self.refinements["equi_dense"]:
            children = self.refinement_equi_dense(children)

        if (self.refinements["region_compaction"]):
            for child in children:
                self.refinement_region_compaction(child)

        node.children.extend(children)
        children.reverse()
        self.nodes_to_cut.pop()
        self.nodes_to_cut.extend(children)
        self.current_node = self.nodes_to_cut[-1]

    def partition_cutsplit(self):
        assert self.current_node is self.root
        from cutsplit import CutSplit
        self._split(self.root, CutSplit(self.rules), "cutsplit")

    def partition_efficuts(self):
        assert self.current_node is self.root
        from efficuts import EffiCuts
        self._split(self.root, EffiCuts(self.rules), "efficuts")

    def _split(self, node, splitter, name):
        key = (name, tuple(str(r) for r in self.rules))
        if key not in SPLIT_CACHE:
            print("Split not cached, recomputing")
            SPLIT_CACHE[key] = [
                p for p in splitter.separate_rules(self.rules) if len(p) > 0
            ]
        parts = SPLIT_CACHE[key]

        parts.sort(key=lambda x: -len(x))
        assert len(self.rules) == sum(len(s) for s in parts)
        print(splitter, [len(s) for s in parts])

        children = []
        for i, p in enumerate(parts):
            c = self.create_node(self.node_count, node.ranges, p,
                                 node.depth + 1, [], i)
            self.node_count += 1
            children.append(c)
        node.action = ("partition", 0, 0)
        self.update_tree(node, children)

    def partition_current_node(self, part_dim, part_size):
        return self.partition_node(self.current_node, part_dimension,
                                   part_size)

    def partition_node(self, node, part_dim, part_size):
        assert part_dim in [0, 1, 2, 3, 4], part_dim
        assert part_size in [0, 1, 2, 3, 4, 5], part_size
        self.depth = max(self.depth, node.depth + 1)
        node.action = ("partition", part_dim, part_size)

        def fits(rule, threshold):
            span = rule.ranges[part_dim * 2 + 1] - rule.ranges[part_dim * 2]
            assert span >= 0, rule
            return span < threshold

        small_rules = []
        big_rules = []
        max_size = [2**32, 2**32, 2**16, 2**16, 2**8][part_dim]
        threshold = max_size * 0.02 * 2**part_size  # 2% ... 64%
        for rule in node.rules:
            if fits(rule, threshold):
                small_rules.append(rule)
            else:
                big_rules.append(rule)

        left_part = list(node.partitions)
        left_part.append((True, part_dim, part_size))
        left = self.create_node(self.node_count, node.ranges, small_rules,
                                node.depth + 1, left_part, None)
        self.node_count += 1
        right_part = list(node.partitions)
        right_part.append((False, part_dim, part_size))
        right = self.create_node(self.node_count, node.ranges, big_rules,
                                 node.depth + 1, right_part, None)
        self.node_count += 1

        children = [left, right]
        self.update_tree(node, children)
        return children

    def cut_current_node(self, cut_dimension, cut_num):
        return self.cut_node(self.current_node, cut_dimension, cut_num)

    def cut_node(self, node, cut_dimension, cut_num):
        self.depth = max(self.depth, node.depth + 1)
        node.action = ("cut", cut_dimension, cut_num)
        range_left = node.ranges[cut_dimension * 2]
        range_right = node.ranges[cut_dimension * 2 + 1]
        range_per_cut = math.ceil((range_right - range_left) / cut_num)

        children = []
        assert cut_num > 0, (cut_dimension, cut_num)
        for i in range(cut_num):
            child_ranges = list(node.ranges)
            child_ranges[cut_dimension * 2] = range_left + i * range_per_cut
            child_ranges[cut_dimension * 2 + 1] = min(
                range_right, range_left + (i + 1) * range_per_cut)

            child_rules = []
            for rule in node.rules:
                if rule.is_intersect(cut_dimension,
                                     child_ranges[cut_dimension * 2],
                                     child_ranges[cut_dimension * 2 + 1]):
                    child_rules.append(rule)

            child = self.create_node(self.node_count, child_ranges,
                                     child_rules, node.depth + 1,
                                     node.partitions, node.manual_partition)
            children.append(child)
            self.node_count += 1

        self.update_tree(node, children)
        return children

    def cut_current_node_multi_dimension(self, cut_dimensions, cut_nums):
        self.depth = max(self.depth, self.current_node.depth + 1)
        node = self.current_node
        node.action = (cut_dimensions, cut_nums)

        range_per_cut = []
        for i in range(len(cut_dimensions)):
            range_left = node.ranges[cut_dimensions[i] * 2]
            range_right = node.ranges[cut_dimensions[i] * 2 + 1]
            cut_num = cut_nums[i]
            range_per_cut.append(
                math.ceil((range_right - range_left) / cut_num))

        cut_index = [0 for i in range(len(cut_dimensions))]
        children = []
        while True:
            # compute child ranges
            child_ranges = list(node.ranges)
            for i in range(len(cut_dimensions)):
                dimension = cut_dimensions[i]
                child_ranges[dimension*2] = node.ranges[dimension*2] + \
                    cut_index[i] * range_per_cut[i]
                child_ranges[dimension * 2 + 1] = min(
                    node.ranges[dimension * 2 + 1], node.ranges[dimension * 2]
                    + (cut_index[i] + 1) * range_per_cut[i])

            # compute child rules
            child_rules = []
            for rule in node.rules:
                if rule.is_intersect_multi_dimension(child_ranges):
                    child_rules.append(rule)

            # create new child
            child = self.create_node(self.node_count, child_ranges,
                                     child_rules, node.depth + 1,
                                     node.partitions, node.manual_partition)
            children.append(child)
            self.node_count += 1

            # update cut index
            cut_index[0] += 1
            i = 0
            while cut_index[i] == cut_nums[i]:
                cut_index[i] = 0
                i += 1
                if i < len(cut_nums):
                    cut_index[i] += 1
                else:
                    break

            if i == len(cut_nums):
                break

        self.update_tree(node, children)
        return children

    def cut_current_node_split(self, cut_dimension, cut_position):
        self.depth = max(self.depth, self.current_node.depth + 1)
        node = self.current_node
        node.action = (cut_dimension, cut_position)
        range_left = node.ranges[cut_dimension * 2]
        range_right = node.ranges[cut_dimension * 2 + 1]
        range_per_cut = cut_position - range_left

        children = []
        for i in range(2):
            child_ranges = node.ranges.copy()
            child_ranges[cut_dimension * 2] = range_left + i * range_per_cut
            child_ranges[cut_dimension * 2 + 1] = min(
                range_right, range_left + (i + 1) * range_per_cut)

            child_rules = []
            for rule in node.rules:
                if rule.is_intersect(cut_dimension,
                                     child_ranges[cut_dimension * 2],
                                     child_ranges[cut_dimension * 2 + 1]):
                    child_rules.append(rule)

            child = self.create_node(self.node_count, child_ranges,
                                     child_rules, node.depth + 1,
                                     node.partitions, node.manual_partition)
            children.append(child)
            self.node_count += 1

        self.update_tree(node, children)
        return children

    def get_next_node(self):
        self.nodes_to_cut.pop()
        if len(self.nodes_to_cut) > 0:
            self.current_node = self.nodes_to_cut[-1]
        else:
            self.current_node = None
        return self.current_node

    def check_contiguous_region(self, node1, node2):
        count = 0
        for i in range(5):
            if node1.ranges[i*2+1] == node2.ranges[i*2] or \
                    node2.ranges[i*2+1] == node1.ranges[i*2]:
                if count == 1:
                    return False
                else:
                    count = 1
            elif node1.ranges[i*2] != node2.ranges[i*2] or \
                    node1.ranges[i*2+1] != node2.ranges[i*2+1]:
                return False
        if count == 0:
            return False
        return True

    def merge_region(self, node1, node2):
        for i in range(5):
            node1.ranges[i * 2] = min(node1.ranges[i * 2], node2.ranges[i * 2])
            node1.ranges[i * 2 + 1] = max(node1.ranges[i * 2 + 1],
                                          node2.ranges[i * 2 + 1])

    def refinement_node_merging(self, nodes):
        while True:
            flag = True
            merged_nodes = [nodes[0]]
            last_node = nodes[0]
            for i in range(1, len(nodes)):
                if self.check_contiguous_region(last_node, nodes[i]):
                    if set(last_node.rules) == set(nodes[i].rules):
                        self.merge_region(last_node, nodes[i])
                        flag = False
                        continue

                merged_nodes.append(nodes[i])
                last_node = nodes[i]

            nodes = merged_nodes
            if flag:
                break

        return nodes

    def refinement_rule_overlay(self, node):
        if len(node.rules) == 0 or len(node.rules) > 500:
            return
        node.rules = node.pruned_rules()

    def refinement_region_compaction(self, node):
        if len(node.rules) == 0:
            return

        new_ranges = list(node.rules[0].ranges)
        for rule in node.rules[1:]:
            for i in range(5):
                new_ranges[i * 2] = min(new_ranges[i * 2], rule.ranges[i * 2])
                new_ranges[i * 2 + 1] = max(new_ranges[i * 2 + 1],
                                            rule.ranges[i * 2 + 1])
        for i in range(5):
            node.ranges[i * 2] = max(new_ranges[i * 2], node.ranges[i * 2])
            node.ranges[i * 2 + 1] = min(new_ranges[i * 2 + 1],
                                         node.ranges[i * 2 + 1])

    def refinement_rule_pushup(self):
        nodes_by_layer = [None for i in range(self.depth)]

        current_layer_nodes = [self.root]
        nodes_by_layer[0] = current_layer_nodes
        for i in range(self.depth - 1):
            next_layer_nodes = []
            for node in current_layer_nodes:
                next_layer_nodes.extend(node.children)
            nodes_by_layer[i + 1] = next_layer_nodes
            current_layer_nodes = next_layer_nodes

        for i in reversed(range(self.depth)):
            for node in nodes_by_layer[i]:
                if len(node.children) == 0:
                    node.pushup_rules = set(node.rules)
                else:
                    node.pushup_rules = set(node.children[0].pushup_rules)
                    for j in range(1, len(node.children)):
                        node.pushup_rules = node.pushup_rules.intersection(
                            node.children[j].pushup_rules)
                    for child in node.children:
                        child.pushup_rules = child.pushup_rules.difference(
                            node.pushup_rules)

    def refinement_equi_dense(self, nodes):
        # try to merge
        nodes_copy = []
        max_rule_count = -1
        for node in nodes:
            nodes_copy.append(
                Node(node.id, list(node.ranges), list(node.rules), node.depth,
                     node.partitions, node.manual_partition))
            max_rule_count = max(max_rule_count, len(node.rules))
        while True:
            flag = True
            merged_nodes = [nodes_copy[0]]
            last_node = nodes_copy[0]
            for i in range(1, len(nodes_copy)):
                if self.check_contiguous_region(last_node, nodes_copy[i]):
                    rules = set(last_node.rules).union(
                        set(nodes_copy[i].rules))
                    if len(rules) < len(last_node.rules) + len(nodes_copy[i].rules) and \
                        len(rules) < max_rule_count:
                        rules = list(rules)
                        rules.sort(key=lambda i: i.priority)
                        last_node.rules = rules
                        self.merge_region(last_node, nodes_copy[i])
                        flag = False
                        continue

                merged_nodes.append(nodes_copy[i])
                last_node = nodes_copy[i]

            nodes_copy = merged_nodes
            if flag:
                break

        # check condition
        if len(nodes_copy) <= 8:
            nodes = nodes_copy
        return nodes

    def compute_result(self):
        if self.refinements["rule_pushup"]:
            self.refinement_rule_pushup()

        # memory space
        # non-leaf: 2 + 16 + 4 * child num
        # leaf: 2 + 16 * rule num
        # details:
        #     header: 2 bytes
        #     region boundary for non-leaf: 16 bytes
        #     each child pointer: 4 bytes
        #     each rule: 16 bytes
        result = {"bytes_per_rule": 0, "memory_access": 0, "update_memory_access": 0,## \
            "num_leaf_node": 0, "num_nonleaf_node": 0, "num_node": 0}
        nodes = [self.root]
        while len(nodes) != 0:
            next_layer_nodes = []
            for node in nodes:
                next_layer_nodes.extend(node.children)

                # compute bytes per rule
                if self.is_leaf(node):
                    result["bytes_per_rule"] += 2 + 16 * len(node.rules)
                    result["num_leaf_node"] += 1
                else:
                    result["bytes_per_rule"] += 2 + 16 + 4 * len(node.children)
                    result["num_nonleaf_node"] += 1

            nodes = next_layer_nodes

        result["memory_access"] = self._compute_memory_access(self.root)
        result["update_memory_access"] = self._compute_update_memory_access(self.root)
        result["bytes_per_rule"] = result["bytes_per_rule"] / len(self.rules)
        result[
            "num_node"] = result["num_leaf_node"] + result["num_nonleaf_node"]
        return result

    def _compute_update_memory_access(self, node):
        if self.is_leaf(node) or not node.children:
            return 1

        if node.is_partition():
            return 1 + max(
                self._compute_update_memory_access(n) for n in node.children)
        else:
            return sum(self._compute_update_memory_access(n) for n in node.children)


    def _compute_memory_access(self, node):
        if self.is_leaf(node) or not node.children:
            return 1

        if node.is_partition():
            return sum(self._compute_memory_access(n) for n in node.children)
        else:
            return 1 + max(
                self._compute_memory_access(n) for n in node.children)

    def get_stats(self):
        widths = []
        dim_stats = []
        nodes = [self.root]
        while len(nodes) != 0 and len(widths) < 30:
            dim = [0] * 5
            next_layer_nodes = []
            for node in nodes:
                next_layer_nodes.extend(node.children)
                if node.action and node.action[0] == "cut":
                    dim[node.action[1]] += 1
            widths.append(len(nodes))
            dim_stats.append(dim)
            nodes = next_layer_nodes
        return {
            "widths": widths,
            "dim_stats": dim_stats,
        }

    def stats_str(self):
        stats = self.get_stats()
        out = "widths" + "," + ",".join(map(str, stats["widths"]))
        out += "\n"
        for i in range(len(stats["dim_stats"][0])):
            out += "dim{}".format(i) + "," + ",".join(
                str(d[i]) for d in stats["dim_stats"])
            out += "\n"
        return out

    def print_stats(self):
        print(self.stats_str())

    def print_layers(self, layer_num=5):
        path = '/content/drive/MyDrive/CIAL/output.txt'
        f = open(path, 'w')
        nodes = [self.root]
        for i in range(layer_num):
            if len(nodes) == 0:
                return
            print("Layer", i, file=f)
            next_layer_nodes = []
            for node in nodes:
                print(node, file=f)                
                next_layer_nodes.extend(node.children)
            nodes = next_layer_nodes

    def __str__(self):
        result = ""
        nodes = [self.root]
        while len(nodes) != 0:
            next_layer_nodes = []
            for node in nodes:
                result += "%d; %s; %s; [" % (node.id, str(node.action),
                                             str(node.ranges))
                for child in node.children:
                    result += str(child.id) + " "
                result += "]\n"
                next_layer_nodes.extend(node.children)
            nodes = next_layer_nodes
        return result
```


partition_time.c 程式碼


```
# 此內容會顯示為程式碼
#define _CRT_SECURE_NO_WARNINGS
#include<stdlib.h>
#include<stdio.h>
#include<string.h>
#include<math.h>
#include<limits.h>

#define SPFAC 64
#define BINTH 16
////////////////////////////////////////////////////////////////////////////////////

static __inline__ unsigned long long rdtsc(void)
{
	unsigned hi, lo;
	__asm__ __volatile__("rdtsc" : "=a"(lo), "=d"(hi));
	return ((unsigned long long)lo) | (((unsigned long long)hi) << 32);
}

enum fiveDim{
	NONE,
	SRCIP,
	DESIP,
	SRCPORT,
	DESPORT,
	PROTOCOL
};
////////////////////////////////////////////////////////////////////////////////////
struct ENTRY{
	unsigned int src_ip;
	unsigned char src_len;
	unsigned int des_ip;
	unsigned char des_len;
	unsigned int src_port_start;
	unsigned int src_port_end;
	unsigned int des_port_start;
	unsigned int des_port_end;
	unsigned int protocol;
};
/////////////////////////////////////////////////////////////////////////////////////
struct PKL{
	unsigned int node_id;
	unsigned int action_type;
	unsigned int dimension;
	unsigned int num_of_bits;
	unsigned int num_of_childs;
	int child_array[64];
};
////////////////////////////////////////////////////////////////////////////////////
struct list{
	unsigned int port;
	struct list *left,*right;
};
typedef struct list node;
typedef node *btrie;
////////////////////////////////////////////////////////////////////////////////////
struct bucket {
	enum fiveDim cut_dim;
	unsigned char bit_length;
	unsigned int *index_array;
	unsigned int arraySize;
	struct bucket *child;
	//record how many bit had check
	unsigned char src_addr_had_check;
	unsigned char des_addr_had_check;
	unsigned char src_port_had_check;
	unsigned char des_port_had_check;
	unsigned char ptc_had_check;
	unsigned int node_id;
};
typedef struct bucket bnode;
typedef bnode *ctrie;
////////////////////////////////////////////////////////////////////////////////////
/*global variables*/
ctrie root;
struct ENTRY *table;
struct ENTRY *query;
struct PKL *pkl;
int num_entry = 0;
int num_pkl = 0;
int num_query = 0;
unsigned long long int begin, end, total = 0;
unsigned long long int *clock;
int counter;
int dim_count[5] = {0,0,0,0,0};
int num_bnode = 0;
////////////////////////////////////////////////////////////////////////////////////
btrie create_node(){
	btrie temp;
	temp=(btrie)malloc(sizeof(node));
	temp->right=NULL;
	temp->left=NULL;
	temp->port=65536;
	return temp;
}
////////////////////////////////////////////////////////////////////////////////////
ctrie create_bnode() {
	ctrie temp;
	temp = (ctrie)malloc(sizeof(bnode));
	temp->cut_dim = NONE;
	temp->bit_length = 0;
	temp->index_array = NULL;
	temp->arraySize = 0;
	temp->child = NULL;
	temp->src_addr_had_check = 0;
	temp->des_addr_had_check = 0;
	temp->src_port_had_check = 0;
	temp->des_port_had_check = 0;
	temp->ptc_had_check = 0;
	temp->node_id = 0; ///update

	return temp;
}
////////////////////////////////////////////////////////////////////////////////////
void read_table(char *str, unsigned int *src_ip, unsigned char *src_len, unsigned int *des_ip, unsigned char *des_len, unsigned int *src_port_start, unsigned int *src_port_end, unsigned int *des_port_start, unsigned int *des_port_end, unsigned int *protocol){
	char tok_space[]="\t";
	char tok_ip[] = "@./ ";
	char tok_port[] = ":\t ";
	char tok_protocol[] = "\tx/ ";
	char buf[200];
	char *sa, *da, *sp, *dp, *pt;

	//source address
	sa = strtok(str, tok_space);
	//destination address
	da = strtok(NULL, tok_space);
	//source port
	sp = strtok(NULL, tok_space);
	//destination address
	dp = strtok(NULL, tok_space);
	//protocol
	pt = strtok(NULL, tok_space);
	//final field
	sprintf(buf, "%s\0", strtok(NULL, tok_space));

	//deal with source ip
	*src_ip = 0;
	sprintf(buf, "%s\0", strtok(sa, tok_ip));
	*src_ip += atoi(buf);
	*src_ip <<= 8;
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*src_ip += atoi(buf);
	*src_ip <<= 8;
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*src_ip += atoi(buf);
	*src_ip <<= 8;
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*src_ip += atoi(buf);
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*src_len = atoi(buf);
	//deal with destination ip
	*des_ip = 0;
	sprintf(buf, "%s\0", strtok(da, tok_ip));
	*des_ip += atoi(buf);
	*des_ip <<= 8;
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*des_ip += atoi(buf);
	*des_ip <<= 8;
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*des_ip += atoi(buf);
	*des_ip <<= 8;
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*des_ip += atoi(buf);
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*des_len = atoi(buf);
	//deal with source port
	sprintf(buf, "%s\0", strtok(sp, tok_port));
	*src_port_start = atoi(buf);
	sprintf(buf, "%s\0", strtok(NULL, tok_port));
	*src_port_end = atoi(buf);
	//deal with destination port
	sprintf(buf, "%s\0", strtok(dp, tok_port));
	*des_port_start = atoi(buf);
	sprintf(buf, "%s\0", strtok(NULL, tok_port));
	*des_port_end = atoi(buf);
	//deal with protocol
	sprintf(buf, "%s\0", strtok(pt, tok_protocol));
	sprintf(buf, "%s\0", strtok(NULL, tok_protocol));
	*protocol = buf[0] >= 'a' ? (buf[0] - 'a' + 10) * 16 : (buf[0] - '0') * 16;
	*protocol += buf[1] >= 'a' ? (buf[1] - 'a' + 10) : (buf[1] - '0');
}
///////////////////////////////////////////////////////////////////////////////////
//string, &node_id, &action_type, &dimension, &num_of_bits,  &child_array
void read_pkl(char *str, unsigned int *node_id, unsigned int *action_type, unsigned int *dimension, unsigned int *num_of_bits, unsigned int *num_of_childs , unsigned int *child_array){
	char tok_space[]=";";
	char tok_ip[] = "(), ";
	char tok_port[] = ":\t ";
	char tok_protocol[] = "\tx/ ";
	char tok_child[] = "[] ";
	char buf[400];
	char *id, *action, *bound, *arr, *temp;
	//int child_array[64];
	int child_count = 0;


	id = strtok(str, tok_space);
	action = strtok(NULL, tok_space);
	bound = strtok(NULL, tok_space);
	arr = strtok(NULL, tok_space);
	//printf("%s\n",id);
	//printf("%s\n",action);
	//printf("%s\n",bound);
	//printf("%s\n",arr);



	//printf("%s\n",arr);
	//printf("len:%d\n",strlen(arr));
	//printf("%s\n",temp);
	//printf("len:%d\n",strlen(temp));

	if(strlen(arr) == 4){
		*num_of_childs = 0;
		//printf("len=0\n");
	}
	else{
		temp = strtok(arr, tok_child);
		//child_array[child_count] = atoi(temp);
		//child_count += 1;
		//printf("%d\n",child_array[0]);
		//temp = strtok(NULL, tok_child);
		//printf("%s\n",temp);
		//printf("%d\n",atoi(temp));
		//printf("len:%d\n",strlen(temp));
		/*child_array[child_count] = atoi(temp);
		child_count += 1;
		temp = strtok(NULL, tok_child);
		printf("%s\n",temp);
		printf("len:%d\n",strlen(temp));*/
		while(atoi(temp) != 0){
			child_array[child_count] = atoi(temp);
			child_count += 1;
			temp = strtok(NULL, tok_child);
			//printf("%d\n",atoi(temp));
			//if(temp==NULL) printf("O\n");
		}
		*num_of_childs = child_count;
	}
	//printf("%d\n---\n",child_array[child_count-1]);
	//printf("%d\n---\n",child_count);
	//printf("%d\n",strlen(temp));
	//printf("%s\n---\n",temp);
	//deal with node id
	*node_id = atoi(id);

	//deal with action type
	*action_type = 0;
	sprintf(buf, "%s\0", strtok(action, tok_ip));
	if(strstr(buf, "cut")){
		*action_type = 1;
	}
	else if(strstr(buf, "partition")){
		*action_type = 2;
	}
	else{
		*action_type = 0;
	}
	//deal with dimension
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*dimension = atoi(buf);
	//printf("%d\n---\n",*dimension);



	//deal with num_of_bits
	sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	*num_of_bits = atoi(buf);
	//printf("%d\n---\n",*num_of_bits);
	//return child_array;

	//*boundary = bound;
	//printf("node id :%d\n---\n",*node_id);
	//printf("%d\n---\n",*num_of_childs);

	//sprintf(buf, "%s\0", strtok(NULL, tok_ip));
	//*child_array = arr;
	//printf("%s\n---\n",arr);

}
////////////////////////////////////////////////////////////////////////////////////
void set_table(char *file_name){
	FILE *fp;
	char string[200];
	unsigned int src_ip;
	unsigned char src_len;
	unsigned int des_ip;
	unsigned char des_len;
	unsigned int src_port_start;
	unsigned int src_port_end;
	unsigned int des_port_start;
	unsigned int des_port_end;
	unsigned int protocol;

	fp=fopen(file_name,"r");
	while(fgets(string,200,fp)!=NULL){
		read_table(string, &src_ip, &src_len, &des_ip, &des_len, &src_port_start, &src_port_end, &des_port_start, &des_port_end, &protocol);
		num_entry++;
	}
	rewind(fp);
	table = (struct ENTRY *)malloc(num_entry * sizeof(struct ENTRY));
	num_entry = 0;
	while(fgets(string,200,fp)!=NULL){
		read_table(string, &src_ip, &src_len, &des_ip, &des_len, &src_port_start, &src_port_end, &des_port_start, &des_port_end, &protocol);
		table[num_entry].src_ip = src_ip;
		table[num_entry].src_len = src_len;
		table[num_entry].des_ip = des_ip;
		table[num_entry].des_len = des_len;
		table[num_entry].src_port_start = src_port_start;
		table[num_entry].src_port_end = src_port_end;
		table[num_entry].des_port_start = des_port_start;
		table[num_entry].des_port_end = des_port_end;
		table[num_entry++].protocol = protocol;
	}
}
////////////////////////////////////////////////////////////////////////////////////
void set_pkl(char *file_name) { //load .pkl file
	FILE *fp;
	char string[2000];
	unsigned int node_id;
	unsigned int action_type;
	unsigned int dimension;
	unsigned int num_of_bits;
	unsigned int num_of_childs;
	unsigned int idx;
	unsigned int max_node_id = 0;
	//char boundary[400];
	int * temp;
	int child_array[64];

	fp = fopen(file_name, "r");
	while (fgets(string, 800, fp) != NULL) {
		read_pkl(string, &node_id, &action_type, &dimension, &num_of_bits, &num_of_childs, child_array);
		if(node_id>max_node_id) max_node_id = node_id;
	}
	//printf("test\n");
	rewind(fp);
	//printf("test1");
	pkl = (struct PKL *)malloc(max_node_id * sizeof(struct PKL));
	//clock = (unsigned long long int *)malloc(num_pkl * sizeof(unsigned long long int));
	//num_pkl = 0;
	while (fgets(string, 800, fp) != NULL) {
		read_pkl(string, &node_id, &action_type, &dimension, &num_of_bits, &num_of_childs, child_array);
		idx = node_id;
		pkl[idx].node_id = node_id;
		pkl[idx].action_type = action_type;
		pkl[idx].dimension = dimension;
		pkl[idx].num_of_childs = num_of_childs;
		//pkl[num_pkl].idx = num_pkl;


		for(int i=0;i<num_of_childs;i++){
			pkl[idx].child_array[i] = *(child_array+i);
			//printf("%d\n",pkl[idx].child_array[i]); ///------------------------testing
		}
		//printf("num_of_childs : %d\n",num_of_childs);
		//printf("node_id : %d\n---\n",idx);
		pkl[idx].num_of_bits = num_of_bits;

		//pkl[num_pkl].child_array = *child_array;
		//printf("%d\n",pkl[idx].node_id);
		//printf("%d\n---\n",pkl[num_pkl].child_array[4]);


		//printf("%d\n---\n",*(child_array+1));
		//printf("%d\n",num_of_childs);
		//printf("%d\n---\n",child_array[num_of_childs-1]);
		//pkl[num_pkl].boundary = boundary;
		//pkl[num_pkl].child_array = child_array;
		//printf("%d\n---\n",num_of_childs);
		//printf("%s\n---\n",child_array);
		//clock[num_pkl++] = 10000000;
	}
	/*
	unsigned int node_id;
	unsigned int action_type;
	unsigned int dimension;
	unsigned int num_of_bits;
	char boundary[400];
	char child_array[400];*/

}
////////////////////////////////////////////////////////////////////////////////////
void set_query(char *file_name) {
	FILE *fp;
	char string[200];
	unsigned int src_ip;
	unsigned char src_len;
	unsigned int des_ip;
	unsigned char des_len;
	unsigned int src_port_start;
	unsigned int src_port_end;
	unsigned int des_port_start;
	unsigned int des_port_end;
	unsigned int protocol;

	fp = fopen(file_name, "r");
	while (fgets(string, 200, fp) != NULL) {
		read_table(string, &src_ip, &src_len, &des_ip, &des_len, &src_port_start, &src_port_end, &des_port_start, &des_port_end, &protocol);
		num_query++;
	}
	rewind(fp);
	query = (struct ENTRY *)malloc(num_query * sizeof(struct ENTRY));
	clock = (unsigned long long int *)malloc(num_query * sizeof(unsigned long long int));
	num_query = 0;
	while (fgets(string, 200, fp) != NULL) {
		read_table(string, &src_ip, &src_len, &des_ip, &des_len, &src_port_start, &src_port_end, &des_port_start, &des_port_end, &protocol);
		query[num_query].src_ip = src_ip;
		query[num_query].src_len = src_len;
		query[num_query].des_ip = des_ip;
		query[num_query].des_len = des_len;
		query[num_query].src_port_start = src_port_start;
		query[num_query].src_port_end = src_port_end;
		query[num_query].des_port_start = des_port_start;
		query[num_query].des_port_end = des_port_end;
		query[num_query].protocol = protocol;
		clock[num_query++] = 10000000;
	}
}
////////////////////////////////////////////////////////////////////////////////////
int addr_tree_addnode(btrie r, unsigned int ip, unsigned char len, unsigned int hop) {
	btrie ptr = r;
	int i;
	for (i = 0; i < len; i++) {
		if (ip&(1 << (31 - i))) {
			if (ptr->right == NULL)
				ptr->right = create_node();
			ptr = ptr->right;
			if ((i == len - 1) && (ptr->port == 65536))
				ptr->port = hop;
		}
		else {
			if (ptr->left == NULL)
				ptr->left = create_node();
			ptr = ptr->left;
			if ((i == len - 1) && (ptr->port == 65536))
				ptr->port = hop;
		}
	}
}
////////////////////////////////////////////////////////////////////////////////////
void port_seg_addnode(btrie* port_seg, unsigned int port_start, unsigned int port_end) {
	btrie ptr = port_seg[port_start];
	int i;
	for (i = 0; i < 16; i++) {
		if (ptr->port == 65536) {
			ptr->port = port_end;
			return;
		}
		else if (port_end > ptr->port) {
			if (ptr->right == NULL)
				ptr->right = create_node();
			ptr = ptr->right;
		}
		else if (port_end < ptr->port) {
			if (ptr->left == NULL)
				ptr->left = create_node();
			ptr = ptr->left;
		}
		else
			return;
	}
	ptr->port = port_end;
}
////////////////////////////////////////////////////////////////////////////////////
void count_tree_distinct(btrie p) {
	if (p == NULL)return;
	count_tree_distinct(p->left);
	count_tree_distinct(p->right);
	if (p->port != 65536)counter++;
}
////////////////////////////////////////////////////////////////////////////////////
void free_tree(btrie p ) {
	if (p == NULL)return;
	free_tree(p->left);
	free_tree(p->right);
	free(p);
}
////////////////////////////////////////////////////////////////////////////////////
void create() {
	int i;
	//initialize root node
	num_bnode++;
	root = create_bnode();
	root->index_array = malloc(sizeof(unsigned int) * num_entry);
	root->arraySize = num_entry;
	//root->node_id = 0;
	//root->idx = 0;
	for (i = 0; i < num_entry; i++)
		root->index_array[i] = i;
}
////////////////////////////////////////////////////////////////////////////////////
void find_cutOfDim(ctrie p) {
	/*int i;
	int index;
	int src_addr_DN, des_addr_DN, src_port_DN, des_port_DN, ptc_DN;
	btrie src_root, des_root, src_port_seg[65536], des_port_seg[65536];
	unsigned int ptc_seg[256];

	//initialize
	src_root = create_node();
	des_root = create_node();
	for (i = 0; i < 65536; i++) {
		src_port_seg[i] = create_node();
		des_port_seg[i] = create_node();
	}
	for (i = 0; i < 256; i++)
		ptc_seg[i] = 0;

	//add node with 5 dim
	for (i = 0; i < p->arraySize; i++) {
		index = p->index_array[i];
		addr_tree_addnode(src_root, table[index].src_ip, table[index].src_len, table[index].src_port_start);
		addr_tree_addnode(des_root, table[index].des_ip, table[index].des_len, table[index].des_port_start);
		port_seg_addnode(src_port_seg, table[index].src_port_start, table[index].src_port_end);
		port_seg_addnode(des_port_seg, table[index].des_port_start, table[index].des_port_end);
		ptc_seg[table[index].protocol]++;
	}

	//count src address distinct number
	counter = 0;
	count_tree_distinct(src_root);
	src_addr_DN = counter;
	//count des address distinct number
	counter = 0;
	count_tree_distinct(des_root);
	des_addr_DN = counter;
	//count src port distinct number
	counter = 0;
	for (i = 0; i < 65536; i++) {
		if (src_port_seg[i]->port != 65536)
			count_tree_distinct(src_port_seg[i]);
	}
	src_port_DN = counter;
	//count des port distinct number
	counter = 0;
	for (i = 0; i < 65536; i++) {
		if (des_port_seg[i]->port != 65536)
			count_tree_distinct(des_port_seg[i]);
	}
	des_port_DN = counter;
	//count ptc distinct number
	counter = 0;
	for (i = 0; i < 256; i++) {
		if (ptc_seg[i] != 0)
			counter++;
	}
	ptc_DN = counter;
*/
	//select max distinct number dim
	//pkl[num_query].dimension
	enum fiveDim max_dim = NONE;
	if(pkl[p->node_id].dimension==0){
		max_dim = SRCIP;
	}
	else if(pkl[p->node_id].dimension==1){
		max_dim = DESIP;
	}
	else if(pkl[p->node_id].dimension==2){
		max_dim = SRCPORT;
	}
	else if(pkl[p->node_id].dimension==3){
		max_dim = DESPORT;
	}
	else{
		max_dim = PROTOCOL;
	}
	/*enum fiveDim max_dim = NONE;
	int max_number = INT_MIN;
	if (src_addr_DN >= des_addr_DN) {
		max_dim = SRCIP;
		max_number = src_addr_DN;
		dim_count[0]++;
	}
	else {
		max_dim = DESIP;
		max_number = des_addr_DN;
		dim_count[1]++;
	}
	if (src_port_DN > max_number) {
		max_dim = SRCPORT;
		max_number = src_port_DN;
		dim_count[2]++;
	}
	if (des_port_DN > max_number) {
		max_dim = DESPORT;
		max_number = des_port_DN;
		dim_count[3]++;
	}
	if (ptc_DN > max_number) {
		max_dim = PROTOCOL;
		max_number = ptc_DN;
		dim_count[4]++;
	}
	//free memory space
	free_tree(src_root);
	free_tree(des_root);
	for (i = 0; i < 65536; i++){
		free_tree(src_port_seg[i]);
		free_tree(des_port_seg[i]);
	}*/
	p->cut_dim = max_dim;
}
////////////////////////////////////////////////////////////////////////////////////
void choose_numOfpart(ctrie ptr) {
	/********************************************************************************
	   We are doing a binary search on the number of cuts to be made at this node(v).
	   When the number of cuts are such that the corresponding memory consumptionestimate becomes more than what is allowed by the space Measure function "spmf()", we end the search.
	   It is possible to do smarter variations of this search algorithm.
	 *******************************************************************************/
	int i, j;
	int cut_length;
	int done = 1;
	int nump;
	unsigned int s_index,e_index;
	int *count_array = NULL;
	int smC;
	int spmf = SPFAC * ptr->arraySize;

	/*for (cut_length = 2; cut_length < 32; cut_length++) {
		nump = pow(2, cut_length);
		if (nump >= sqrt(ptr->arraySize))
			break;
	}*/

	//brute force to find best cut of part
	/*
	while(done){
		smC = 0;
		count_array = malloc(sizeof(int)*nump);
		for (i = 0; i < nump; i++)
			count_array[i] = 0;
		//select cut dim
		if (ptr->cut_dim == SRCIP) {
			for (j = 0; j < ptr->arraySize; j++) {
				if (cut_length > table[ptr->index_array[j]].src_len + ptr->src_addr_had_check) {
					s_index = table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check  >>  (32 - table[ptr->index_array[j]].src_len) << (cut_length - table[ptr->index_array[j]].src_len);
					e_index = ((table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check) | (0xFFFFFFFF >> table[ptr->index_array[j]].src_len)) >> (32 - cut_length);
					for (i = s_index; i <= e_index; i++)
						count_array[i]++;
				}
				else {
					s_index = table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check >> (32 - cut_length);
					count_array[s_index]++;
				}
			}
		}
		else if (ptr->cut_dim == DESIP) {
			for (j = 0; j < ptr->arraySize; j++) {
				if (cut_length > table[ptr->index_array[j]].des_len + ptr->des_addr_had_check) {
					s_index = table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check >> (32 - table[ptr->index_array[j]].des_len) << (cut_length - table[ptr->index_array[j]].des_len);
					e_index = ((table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check) | (0xFFFFFFFF >> table[ptr->index_array[j]].des_len)) >> (32 - cut_length);
					for (i = s_index; i <= e_index; i++)
						count_array[i]++;
				}
				else {
					s_index = table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check >> (32 - cut_length);
					count_array[s_index]++;
				}
			}
		}
		else if (ptr->cut_dim == SRCPORT) {
			for (j = 0; j < ptr->arraySize; j++) {
				s_index = table[ptr->index_array[j]].src_port_start << ptr->src_port_had_check >> (32 - cut_length);
				e_index = table[ptr->index_array[j]].src_port_end << ptr->src_port_had_check >> (32 - cut_length);
				for (i = s_index; i <= e_index; i++)
					count_array[i]++;
			}
		}
		else if (ptr->cut_dim == DESPORT) {
			for (j = 0; j < ptr->arraySize; j++) {
				s_index = table[ptr->index_array[j]].des_port_start << ptr->des_port_had_check >> (32 - cut_length);
				e_index = table[ptr->index_array[j]].des_port_end << ptr->des_port_had_check >> (32 - cut_length);
				for (i = s_index; i <= e_index; i++)
					count_array[i]++;
			}
		}
		else if (ptr->cut_dim == PROTOCOL) {
			for (j = 0; j < ptr->arraySize; j++) {
				s_index = table[ptr->index_array[j]].protocol << ptr->ptc_had_check >> (32 - cut_length);
				count_array[s_index]++;
			}
		}
		else
			printf("******************ERROR*********************");

		//smC += number of partitions colliding with rule r;
		for (int i = 0; i < nump; i++)
			smC += count_array[i];
		smC += nump;
		//check if comply formula
		if (smC < spmf) {
			nump *= 2;
			cut_length++;
			free(count_array);
		}
		// break
		else
			done = 0;
	}*/

	//redo last time operation-----------------------
	/*nump /= 2;
	cut_length--;
	free(count_array);*/
	nump = pkl[ptr->node_id].num_of_bits;
	//printf("%d\n",ptr->node_id);
	//printf("%d\n",nump);
	cut_length = log2(nump);
	//printf("%d---\n",cut_length);
	count_array = malloc(sizeof(int)*nump);

	for (i = 0; i < nump; i++)
		count_array[i] = 0;
	//select cut dim
	if (ptr->cut_dim == SRCIP) {
		for (j = 0; j < ptr->arraySize; j++) {
			if (cut_length > table[ptr->index_array[j]].src_len + ptr->src_addr_had_check) {
				s_index = table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check >> (32 - table[ptr->index_array[j]].src_len) << (cut_length - table[ptr->index_array[j]].src_len);
				e_index = ((table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check) | (0xFFFFFFFF >> table[ptr->index_array[j]].src_len)) >> (32 - cut_length);
				for (i = s_index; i <= e_index; i++)
					count_array[i]++;
			}
			else {
				s_index = table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check >> (32 - cut_length);
				count_array[s_index]++;
			}
		}
	}
	else if (ptr->cut_dim == DESIP) {
		for (j = 0; j < ptr->arraySize; j++) {
			if (cut_length > table[ptr->index_array[j]].des_len + ptr->des_addr_had_check) {
				s_index = table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check >> (32 - table[ptr->index_array[j]].des_len) << (cut_length - table[ptr->index_array[j]].des_len);
				e_index = ((table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check) | (0xFFFFFFFF >> table[ptr->index_array[j]].des_len)) >> (32 - cut_length);
				for (i = s_index; i <= e_index; i++)
					count_array[i]++;
			}
			else {
				s_index = table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check >> (32 - cut_length);
				count_array[s_index]++;
			}
		}
	}
	else if (ptr->cut_dim == SRCPORT) {
		for (j = 0; j < ptr->arraySize; j++) {
			s_index = table[ptr->index_array[j]].src_port_start << ptr->src_port_had_check >> (32 - cut_length);
			e_index = table[ptr->index_array[j]].src_port_end << ptr->src_port_had_check >> (32 - cut_length);
			for (i = s_index; i <= e_index; i++)
				count_array[i]++;
		}
	}
	else if (ptr->cut_dim == DESPORT) {
		for (j = 0; j < ptr->arraySize; j++) {
			s_index = table[ptr->index_array[j]].des_port_start << ptr->des_port_had_check >> (32 - cut_length);
			e_index = table[ptr->index_array[j]].des_port_end << ptr->des_port_had_check >> (32 - cut_length);
			for (i = s_index; i <= e_index; i++)
				count_array[i]++;
		}
	}
	else if (ptr->cut_dim == PROTOCOL) {
		for (j = 0; j < ptr->arraySize; j++) {
			s_index = table[ptr->index_array[j]].protocol << ptr->ptc_had_check >> (32 - cut_length);
			count_array[s_index]++;
		}
	}
	else
		printf("******************ERROR*********************");

	//set this ctrie node info
	ptr->bit_length = cut_length;
	ptr->child = (ctrie)malloc(sizeof(bnode)*nump);
	int idx_child=0;
	int flag = 1;
	for (i = 0; i < nump; i++) {
		num_bnode++;
		ptr->child[i].arraySize = 0;
		ptr->child[i].bit_length = 0;
		ptr->child[i].child = NULL;
		ptr->child[i].cut_dim = NONE;
		ptr->child[i].src_addr_had_check = ptr->src_addr_had_check;
		ptr->child[i].des_addr_had_check = ptr->des_addr_had_check;
		ptr->child[i].src_port_had_check = ptr->src_port_had_check;
		ptr->child[i].des_port_had_check = ptr->des_port_had_check;
		ptr->child[i].ptc_had_check = ptr->ptc_had_check;
		//pkl[num_query].child_array[i]
		if ((count_array[i] != 0)||(flag==1)){
			if(count_array[i] != 0){   //first null node is in child array
				flag = 1;
			}
			else flag = 0;
			ptr->child[i].index_array = malloc(sizeof(unsigned int) * count_array[i]);
			ptr->child[i].node_id = pkl[ptr->node_id].child_array[idx_child];////////bug----------------------------------------------------------
			idx_child++;
			//printf("%d\n",pkl[ptr->node_id].child_array[0]);
			//printf("%d\n",count_array[i]);
			//printf("%d\n",idx_child);
			//printf("%d\n---\n",ptr->child[i].node_id);
		}
		else{
			ptr->child[i].index_array = NULL;
			ptr->child[i].node_id =-1;
		}
	}

	free(count_array);

	//distribution to child's bucket
	if (ptr->cut_dim == SRCIP) {
		for (j = 0; j < ptr->arraySize; j++) {
			if (cut_length > table[ptr->index_array[j]].src_len + ptr->src_addr_had_check) {
				s_index = table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check >> (32 - table[ptr->index_array[j]].src_len) << (cut_length - table[ptr->index_array[j]].src_len);
				e_index = ((table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check) | (0xFFFFFFFF >> table[ptr->index_array[j]].src_len)) >> (32 - cut_length);
				for (i = s_index; i <= e_index; i++) {
					ptr->child[i].index_array[ptr->child[i].arraySize++] = ptr->index_array[j];
					ptr->child[i].src_addr_had_check = ptr->src_addr_had_check + ptr->bit_length;
				}
			}
			else {
				s_index = table[ptr->index_array[j]].src_ip << ptr->src_addr_had_check >> (32 - cut_length);
				ptr->child[s_index].index_array[ptr->child[s_index].arraySize++] = ptr->index_array[j];
				ptr->child[s_index].src_addr_had_check = ptr->src_addr_had_check + ptr->bit_length;
			}
		}
	}
	else if (ptr->cut_dim == DESIP) {
		for (j = 0; j < ptr->arraySize; j++) {
			if (cut_length > table[ptr->index_array[j]].des_len + ptr->des_addr_had_check) {
				s_index = table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check >> (32 - table[ptr->index_array[j]].des_len) << (cut_length - table[ptr->index_array[j]].des_len);
				e_index = ((table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check) | (0xFFFFFFFF >> table[ptr->index_array[j]].des_len)) >> (32 - cut_length);
				for (i = s_index; i <= e_index; i++) {
					ptr->child[i].index_array[ptr->child[i].arraySize++] = ptr->index_array[j];
					ptr->child[i].des_addr_had_check = ptr->des_addr_had_check + ptr->bit_length;
				}
			}
			else {
				s_index = table[ptr->index_array[j]].des_ip << ptr->des_addr_had_check >> (32 - cut_length);
				ptr->child[s_index].index_array[ptr->child[s_index].arraySize++] = ptr->index_array[j];
				ptr->child[s_index].des_addr_had_check = ptr->des_addr_had_check + ptr->bit_length;
			}
		}
	}
	else if (ptr->cut_dim == SRCPORT) {
		for (j = 0; j < ptr->arraySize; j++) {
			s_index = table[ptr->index_array[j]].src_port_start << ptr->src_port_had_check >> (32 - cut_length);
			e_index = table[ptr->index_array[j]].src_port_end << ptr->src_port_had_check >> (32 - cut_length);
			for (i = s_index; i <= e_index; i++) {
				ptr->child[i].index_array[ptr->child[i].arraySize++] = ptr->index_array[j];
				ptr->child[i].src_port_had_check = ptr->src_port_had_check + ptr->bit_length;
			}
		}
	}
	else if (ptr->cut_dim == DESPORT) {
		for (j = 0; j < ptr->arraySize; j++) {
			s_index = table[ptr->index_array[j]].des_port_start << ptr->des_port_had_check >> (32 - cut_length);
			e_index = table[ptr->index_array[j]].des_port_end << ptr->des_port_had_check >> (32 - cut_length);
			for (i = s_index; i <= e_index; i++) {
				ptr->child[i].index_array[ptr->child[i].arraySize++] = ptr->index_array[j];
				ptr->child[i].des_port_had_check = ptr->des_port_had_check + ptr->bit_length;
			}
		}
	}
	else if (ptr->cut_dim == PROTOCOL) {
		for (j = 0; j < ptr->arraySize; j++) {
			i = table[ptr->index_array[j]].protocol << ptr->ptc_had_check >> (32 - cut_length);
			ptr->child[i].index_array[ptr->child[i].arraySize++] = ptr->index_array[j];
			ptr->child[i].ptc_had_check = ptr->ptc_had_check + ptr->bit_length;
		}
	}
}
////////////////////////////////////////////////////////////////////////////////////
void partition(ctrie ptr) {
	//讀threshold
	//根據dimension做largeness partition

	int part_size,i,j;
	int *count_array = NULL;
	int cut_length=0;
	unsigned int s_index,e_index;
	//int max_size = [2**32, 2**32, 2**16, 2**16, 2**8];
	float partion_threshold[6] = {0.02,0.04,0.08,0.16,0.32,0.64};
	float threshold;
	int range;
	part_size = pkl[ptr->node_id].num_of_bits;

	count_array = malloc(sizeof(int)*2);
	for (i = 0; i < 2; i++)
		count_array[i] = 0;

	if (ptr->cut_dim == SRCIP) {
		threshold = pow(2,32) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = pow(2,32)-table[ptr->index_array[j]].src_ip;
			if(range>threshold) count_array[1]++;
			else count_array[0]++;
		}
	}
	else if (ptr->cut_dim == DESIP) {
		threshold = pow(2,32) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = pow(2,32)-table[ptr->index_array[j]].des_ip;
			if(range>threshold) count_array[1]++;
			else count_array[0]++;
		}
	}
	else if (ptr->cut_dim == SRCPORT) {
		threshold = pow(2,16) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = table[ptr->index_array[j]].src_port_end - table[ptr->index_array[j]].src_port_start;
			if(range>threshold) count_array[1]++;
			else count_array[0]++;
		}
	}
	else if (ptr->cut_dim == DESPORT) {
		threshold = pow(2,16) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = table[ptr->index_array[j]].des_port_end - table[ptr->index_array[j]].des_port_start;
			if(range>threshold) count_array[1]++;
			else count_array[0]++;
		}
	}
	else if (ptr->cut_dim == PROTOCOL) {
		threshold = pow(2,8) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = pow(2,8)-table[ptr->index_array[j]].protocol;
			if(range>threshold) count_array[1]++;
			else count_array[0]++;
		}
	}
	else
		printf("******************ERROR*********************");

	ptr->bit_length = cut_length;
	ptr->child = (ctrie)malloc(sizeof(bnode)*2);
	int idx_child=0;
	//int flag = 1;
	for (i = 0; i < 2; i++) {
		num_bnode++;
		ptr->child[i].arraySize = 0;
		ptr->child[i].bit_length = 0;
		ptr->child[i].child = NULL;
		ptr->child[i].cut_dim = NONE;
		ptr->child[i].src_addr_had_check = ptr->src_addr_had_check;
		ptr->child[i].des_addr_had_check = ptr->des_addr_had_check;
		ptr->child[i].src_port_had_check = ptr->src_port_had_check;
		ptr->child[i].des_port_had_check = ptr->des_port_had_check;
		ptr->child[i].ptc_had_check = ptr->ptc_had_check;
		//pkl[num_query].child_array[i]
		if ((count_array[i] != 0)){

			ptr->child[i].index_array = malloc(sizeof(unsigned int) * count_array[i]);
			ptr->child[i].node_id = pkl[ptr->node_id].child_array[idx_child];////////bug----------------------------------------------------------
			idx_child++;
			//printf("%d\n",pkl[ptr->node_id].child_array[0]);
			//printf("%d\n",count_array[i]);
			//printf("%d\n",idx_child);
			//printf("%d\n---\n",ptr->child[i].node_id);
		}
		else{
			ptr->child[i].index_array = NULL;
			ptr->child[i].node_id =-1;
		}
	}

	free(count_array);
	//printf("test\n");
	if (ptr->cut_dim == SRCIP) {
		threshold = pow(2,32) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = pow(2,32)-table[ptr->index_array[j]].src_ip;
			if(range>threshold){
				ptr->child[1].index_array[ptr->child[1].arraySize++] = ptr->index_array[j];
				ptr->child[1].src_addr_had_check = ptr->src_addr_had_check;
			}
			else{
				ptr->child[0].index_array[ptr->child[0].arraySize++] = ptr->index_array[j];
				ptr->child[0].src_addr_had_check = ptr->src_addr_had_check;
			}
		}
	}
	else if (ptr->cut_dim == DESIP) {
		threshold = pow(2,32) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = pow(2,32)-table[ptr->index_array[j]].des_ip;
			if(range>threshold){
				ptr->child[1].index_array[ptr->child[1].arraySize++] = ptr->index_array[j];
				ptr->child[1].des_addr_had_check = ptr->des_addr_had_check;
			}
			else{
				ptr->child[0].index_array[ptr->child[0].arraySize++] = ptr->index_array[j];
				ptr->child[0].des_addr_had_check = ptr->des_addr_had_check;
			}
		}
	}
	else if (ptr->cut_dim == SRCPORT) {
		threshold = pow(2,16) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = table[ptr->index_array[j]].src_port_end - table[ptr->index_array[j]].src_port_start;
			if(range>threshold){
				ptr->child[1].index_array[ptr->child[1].arraySize++] = ptr->index_array[j];
				ptr->child[1].src_port_had_check = ptr->src_port_had_check;
			}
			else{
				ptr->child[0].index_array[ptr->child[0].arraySize++] = ptr->index_array[j];
				ptr->child[0].src_port_had_check = ptr->src_port_had_check;
			}
		}
	}
	else if (ptr->cut_dim == DESPORT) {
		threshold = pow(2,16) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = table[ptr->index_array[j]].des_port_end - table[ptr->index_array[j]].des_port_start;
			if(range>threshold){
				ptr->child[1].index_array[ptr->child[1].arraySize++] = ptr->index_array[j];
				ptr->child[1].des_port_had_check = ptr->des_port_had_check;
			}
			else{
				ptr->child[0].index_array[ptr->child[0].arraySize++] = ptr->index_array[j];
				ptr->child[0].des_port_had_check = ptr->des_port_had_check;
			}
		}
	}
	else if (ptr->cut_dim == PROTOCOL) {
		threshold = pow(2,8) * partion_threshold[part_size];
		for (j = 0; j < ptr->arraySize; j++) {
			range = pow(2,8)-table[ptr->index_array[j]].protocol;
			if(range>threshold){
				ptr->child[1].index_array[ptr->child[1].arraySize++] = ptr->index_array[j];
				ptr->child[1].ptc_had_check = ptr->ptc_had_check;
			}
			else{
				ptr->child[0].index_array[ptr->child[0].arraySize++] = ptr->index_array[j];
				ptr->child[0].ptc_had_check = ptr->ptc_had_check;
			}
		}
	}

}


////////////////////////////////////////////////////////////////////////////////////
void cut(ctrie p) {
	if (p->arraySize <= BINTH)
		return;

	ctrie ptr = p;
	//printf("node: %d\n",ptr->node_id);
	if(pkl[ptr->node_id].action_type==0) return;

	find_cutOfDim(ptr);
	if (ptr->cut_dim == SRCIP && ptr->src_addr_had_check >= 32)
		return;
	else if (ptr->cut_dim == DESIP && ptr->des_addr_had_check >= 32)
		return;
	else if (ptr->cut_dim == SRCPORT && ptr->src_port_had_check >= 16)
		return;
	else if (ptr->cut_dim == DESPORT && ptr->des_port_had_check >= 16)
		return;
	else if (ptr->cut_dim == PROTOCOL && ptr->ptc_had_check >= 8)
		return;

	if(pkl[ptr->node_id].action_type==2){ //partition
		partition(ptr);
	}
	else{
		choose_numOfpart(ptr);
	}
	//printf("node: %d\n",ptr->node_id);
	//ptr->idx = pkl[ptr->idx].child_array[idx_child];//////////////////////bug


	for (int i = 0; i < pow(2,ptr->bit_length); i++) {
		if (ptr->child[i].arraySize > 0)
			cut(&ptr->child[i]);
	}
}
////////////////////////////////////////////////////////////////////////////////////
void search(unsigned int src_ip, unsigned int des_ip, unsigned int src_port, unsigned int des_port, unsigned int protocol) {
	ctrie ptr = root;
	//printf("test\n");
	int find = 0;
	int target = -1;
	unsigned int index;
	//printf("%ud ",src_ip);
	//printf("%ud ",des_ip);
	//printf("%d ",src_port);
	//printf("%d ",des_port);
	//printf("%d\n---\n ",protocol);
	while (find != 1 && ptr->node_id!=-1) {
		//printf("type: %d, dim: %d\n",pkl[ptr->node_id].action_type,ptr->cut_dim);
		if(ptr->node_id==-1){
			find = 1;
			break;
		}
		else if(pkl[ptr->node_id].action_type==2){
			if (ptr->cut_dim == SRCIP) {
				if((ptr->child[0].index_array)!=NULL){
					ptr = &(ptr->child[0]);
				}
				else if((ptr->child[1].index_array)!=NULL){
					ptr = &(ptr->child[1]);
				}else break;
			}
			else if (ptr->cut_dim == DESIP) {
				if((ptr->child[0].index_array)!=NULL){
					ptr = &(ptr->child[0]);
				}
				else ptr = &(ptr->child[1]);
			}
			else if (ptr->cut_dim == SRCPORT) {
				if((ptr->child[0].index_array)!=NULL){
					ptr = &(ptr->child[0]);
				}
				else ptr = &(ptr->child[1]);
			}
			else if (ptr->cut_dim == DESPORT) {
				if((ptr->child[0].index_array)!=NULL){
					ptr = &(ptr->child[0]);
				}
				else ptr = &(ptr->child[1]);
			}
			else if (ptr->cut_dim == PROTOCOL) {
				if((ptr->child[0].index_array)!=NULL){
					ptr = &(ptr->child[0]);
				}
				else if((ptr->child[1].index_array)!=NULL){
					ptr = &(ptr->child[1]);
				}else break;
			}
			else if (ptr->cut_dim == NONE) {
				for (int i = 0; i < ptr->arraySize; i++) {
					if (table[ptr->index_array[i]].src_ip == src_ip && table[ptr->index_array[i]].des_ip == des_ip)
						if (table[ptr->index_array[i]].src_port_start <= src_port && table[ptr->index_array[i]].src_port_end >= src_port && table[ptr->index_array[i]].des_port_start <= des_port && table[ptr->index_array[i]].des_port_end >= des_port && table[ptr->index_array[i]].protocol == protocol) {
							target = table[ptr->index_array[i]].des_ip;
							find = 1;
							break;
						}
				//else break;
				}

				break;
			}
			//else break;
		}
		else{
			if (ptr->cut_dim == SRCIP) {
				index = src_ip << ptr->src_addr_had_check >> (32 - ptr->bit_length);
				ptr = &(ptr->child[index]);
			}
			else if (ptr->cut_dim == DESIP) {
				index = des_ip << ptr->des_addr_had_check >> (32 - ptr->bit_length);
				//printf("%d\n",index);
				//printf("%d\n",des_ip);
				//printf("%d\n",ptr->des_addr_had_check);
				//printf("%d\n",(32 - ptr->bit_length));
				//printf("%d\n",ptr->arraySize);
				ptr = &(ptr->child[index]);

				//printf("testtest\n");
				//printf("%d\n",ptr->node_id);

			}
			else if (ptr->cut_dim == SRCPORT) {
				index = src_port << ptr->src_port_had_check >> (32 - ptr->bit_length);
				ptr = &(ptr->child[index]);
			}
			else if (ptr->cut_dim == DESPORT) {
				index = des_port << ptr->des_port_had_check >> (32 - ptr->bit_length);
				ptr = &(ptr->child[index]);
			}
			else if (ptr->cut_dim == PROTOCOL) {
				index = protocol << ptr->ptc_had_check >> (32 - ptr->bit_length);
				ptr = &(ptr->child[index]);
			}
			else if (ptr->cut_dim == NONE) {
				for (int i = 0; i < ptr->arraySize; i++) {
					if (table[ptr->index_array[i]].src_ip == src_ip && table[ptr->index_array[i]].des_ip == des_ip)
						if (table[ptr->index_array[i]].src_port_start <= src_port && table[ptr->index_array[i]].src_port_end >= src_port && table[ptr->index_array[i]].des_port_start <= des_port && table[ptr->index_array[i]].des_port_end >= des_port && table[ptr->index_array[i]].protocol == protocol) {
							target = table[ptr->index_array[i]].des_ip;
							find = 1;
							break;
						}
					//else break;///不知為啥有的搜不到
				//else break;
				}

				break;
			}
		}
	}

	/*if (target == -1)
		printf("not find\n");
	else
		printf("%x\n", target);
	*/
}
////////////////////////////////////////////////////////////////////////////////////
void CountClock()
{
	unsigned int i;
	unsigned int* NumCntClock = (unsigned int*)malloc(50 * sizeof(unsigned int));
	for (i = 0; i < 50; i++) NumCntClock[i] = 0;
	unsigned long long MinClock = 10000000, MaxClock = 0;
	for (i = 0; i < num_query; i++)
	{
		if (clock[i] > MaxClock) MaxClock = clock[i];
		if (clock[i] < MinClock) MinClock = clock[i];
		if (clock[i] / 100 < 50) NumCntClock[clock[i] / 100]++;
		else NumCntClock[49]++;
	}
	printf("(MaxClock, MinClock) = (%5llu, %5llu)\n", MaxClock, MinClock);

	for (i = 0; i < 50; i++)
	{
		printf("%d\n", NumCntClock[i]);
	}
	return;
}
////////////////////////////////////////////////////////////////////////////////////
void shuffle(struct ENTRY *array, int n) {
	srand((unsigned)time(NULL));
	struct ENTRY *temp = (struct ENTRY *)malloc(sizeof(struct ENTRY));

	for (int i = 0; i < n - 1; i++) {
		size_t j = i + rand() / (RAND_MAX / (n - i) + 1);
		temp->src_ip = array[j].src_ip;
		temp->des_ip = array[j].des_ip;
		temp->src_port_start = array[j].src_port_start;
		temp->src_port_end = array[j].src_port_end;
		temp->des_port_start = array[j].des_port_start;
		temp->des_port_end = array[j].des_port_end;
		temp->src_len = array[j].src_len;
		temp->des_len = array[j].des_len;
		temp->protocol = array[j].protocol;
	}
}
////////////////////////////////////////////////////////////////////////////////////
int main(int argc,char *argv[]){
	int i,j;
	int cut_dim;
	char filename[50] = "8k.txt";
	set_table(argv[1]);
	//printf("test\n");
	set_pkl(argv[2]);
	//printf("test\n");
	set_query(argv[3]);
	//printf("test\n");
	//begin = rdtsc();

	create();
	cut(root);

	//end = rdtsc();
	//printf("Avg. Build Time: %llu\n", (end - begin) / num_entry);
	printf("SPFAC:%d\n", SPFAC);
	printf("BINTH:%d\n", BINTH);
	printf("num of entry:%d\n", num_entry);
	printf("num of bnode : %d\n", num_bnode);
	//printf("dim count:\n");
	//for (i = 0; i < 5; i++)
		//printf("	%d\n", dim_count[i]);

	shuffle(query, num_query);
	for (j = 0; j < 100; j++) {
		for (i = 0; i < num_query; i++) {
			begin = rdtsc();
			//printf("test\n");
			//search(table[i].src_ip, table[i].des_ip, table[i].src_port_start, table[i].des_port_start, table[i].protocol);
			search(query[i].src_ip, query[i].des_ip, query[i].src_port_start, query[i].des_port_start, query[i].protocol);
			end = rdtsc();
			//printf("num :%d\n",i);
			if (clock[i] > (end - begin))
				clock[i] = (end - begin);
		}
	}
	total = 0;
	for (j = 0; j < num_query; j++){
		total += clock[j];
		//printf("%d\n",clock[j]);
	}

	printf("Avg. Search: %lld\n", total / num_query);
	CountClock();

	return 0;
}
```

